In [1]:
# Quadratic Programming Formulation:
#
#       min 1/2*x'*P*x+q'*x
# s.t. G*x <= h
#      A*x == b
#     
# 


In [2]:
from cvxopt import matrix, solvers
import numpy as np
import pandas as pd
import scipy.optimize as opt
import matplotlib.pyplot as plt
%matplotlib inline

print (help(solvers.qp)) ##Help document of quadratic programming solver


Help on function qp in module cvxopt.coneprog:

qp(P, q, G=None, h=None, A=None, b=None, solver=None, kktsolver=None, initvals=None, **kwargs)
    Solves a quadratic program
    
        minimize    (1/2)*x'*P*x + q'*x
        subject to  G*x <= h
                    A*x = b.
    
    
    Input arguments.
    
        P is a n x n dense or sparse 'd' matrix with the lower triangular
        part of P stored in the lower triangle.  Must be positive
        semidefinite.
    
        q is an n x 1 dense 'd' matrix.
    
        G is an m x n dense or sparse 'd' matrix.
    
        h is an m x 1 dense 'd' matrix.
    
        A is a p x n dense or sparse 'd' matrix.
    
        b is a p x 1 dense 'd' matrix or None.
    
        solver is None or 'mosek'.
    
        The default values for G, h, A and b are empty matrices with
        zero rows.
    
    
    Output arguments (default solver).
    
        Returns a dictionary with keys 'status', 'x', 's', 'y', 'z',
        'primal ob

In [3]:
##Objective function: minimize 3*p^2-190*p

##Constraints:
##p<=50
##-p<=-20

P_pricing = matrix([[3*2]],tc='d') ##The quadratic term
q_pricing = matrix([[-190]],tc='d') ##The linear term
G_pricing = matrix([[1,-1]],tc='d') ##G matrix
h_pricing = matrix([[50,-20]],tc='d') ##h vector
A_pricing = matrix(1.,(0,1)) ##A matrix
b_pricing = matrix(1.,(0,1)) ##b vector





In [4]:
print(P_pricing)
print(q_pricing)
print(G_pricing)
print(h_pricing)

[ 6.00e+00]

[-1.90e+02]

[ 1.00e+00]
[-1.00e+00]

[ 5.00e+01]
[-2.00e+01]



In [5]:
pricing = solvers.qp(P_pricing, q_pricing, G_pricing, h_pricing, A_pricing,b_pricing)

     pcost       dcost       gap    pres   dres
 0: -3.0063e+03 -3.0988e+03  9e+01  0e+00  4e-16
 1: -3.0083e+03 -3.0098e+03  2e+00  1e-16  1e-16
 2: -3.0083e+03 -3.0083e+03  2e-02  1e-16  3e-17
 3: -3.0083e+03 -3.0083e+03  2e-04  7e-17  7e-17
Optimal solution found.


In [6]:
print(pricing['x'])

[ 3.17e+01]



In [7]:
### Markowitz Portfolio Optimization Problem
###Import the data


portfolio = pd.read_csv("Portfolio.csv")

portfolio_data = np.array(portfolio)




In [8]:
##Head of this data frame

portfolio.head()

,AAPL,AMZN,DIS,WFM,WMT
0,0.010116,-0.002786,-0.002492,0.015754,0.017394
1,0.013853,-0.001294,0.011291,-0.010133,0.012405
2,0.026565,0.005810,-0.004193,-0.009075,0.014259
3,0.023964,0.003714,0.005066,-0.000400,0.009182
4,0.010041,0.017356,0.016711,0.015587,-0.001892


In [9]:
##Correlation between the monthly return of APPL and AMZN

np.corrcoef(portfolio_data[:,0], portfolio_data[:,1])

array([[1.        , 0.16014092],
       [0.16014092, 1.        ]])

In [10]:
##Correlation Matrix of different stocks
portfolio.corr()

,AAPL,AMZN,DIS,WFM,WMT
AAPL,1.000000,0.160141,0.163068,-0.259636,0.398971
AMZN,0.160141,1.000000,0.029251,0.272152,-0.192858
DIS,0.163068,0.029251,1.000000,0.173332,0.123564
WFM,-0.259636,0.272152,0.173332,1.000000,0.125003
WMT,0.398971,-0.192858,0.123564,0.125003,1.000000


In [11]:
## Construct the correlation matrix

Rho = np.array(portfolio.corr())

## The last asset is the riskless bond, whose return has zero correlation with all other five stocks

z = np.zeros((1,5))

Rho = np.concatenate((Rho, z), axis=0)

z = np.zeros((6,1))

Rho = np.concatenate((Rho, z), axis=1)

Rho[5,5] = 1

print(Rho)


[[ 1.          0.16014092  0.16306802 -0.25963625  0.39897098  0.        ]
 [ 0.16014092  1.          0.02925135  0.27215205 -0.19285793  0.        ]
 [ 0.16306802  0.02925135  1.          0.17333236  0.12356358  0.        ]
 [-0.25963625  0.27215205  0.17333236  1.          0.12500304  0.        ]
 [ 0.39897098 -0.19285793  0.12356358  0.12500304  1.          0.        ]
 [ 0.          0.          0.          0.          0.          1.        ]]


In [12]:
#Covariance matrix of different assets
portfolio.cov()

,AAPL,AMZN,DIS,WFM,WMT
AAPL,0.000130,0.000016,0.000017,-0.000025,0.000029
AMZN,0.000016,0.000073,0.000002,0.000019,-0.000011
DIS,0.000017,0.000002,0.000084,0.000013,0.000007
WFM,-0.000025,0.000019,0.000013,0.000069,0.000007
WMT,0.000029,-0.000011,0.000007,0.000007,0.000041


In [13]:
#Construct the covariance matrix of 6 assets


Cov = np.array(portfolio.cov())

z = np.zeros((1,5))

Cov = np.concatenate((Cov, z), axis=0)

z = np.zeros((6,1))


Cov = np.concatenate((Cov, z), axis=1)

Cov = Cov*12  ##Transform monthly return covariance into yearly return covariance

print(Cov)

[[ 1.55520821e-03  1.86673885e-04  2.04067585e-04 -2.95538641e-04
   3.49881153e-04  0.00000000e+00]
 [ 1.86673885e-04  8.73723380e-04  2.74374604e-05  2.32195212e-04
  -1.26767944e-04  0.00000000e+00]
 [ 2.04067585e-04  2.74374604e-05  1.00698244e-03  1.58761530e-04
   8.71939580e-05  0.00000000e+00]
 [-2.95538641e-04  2.32195212e-04  1.58761530e-04  8.33123432e-04
   8.02343275e-05  0.00000000e+00]
 [ 3.49881153e-04 -1.26767944e-04  8.71939580e-05  8.02343275e-05
   4.94504155e-04  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00]]


In [14]:
##Estimate the mean annual return
Mean = np.array(portfolio.mean())
Mean = np.reshape(Mean,(-1,1))
z = np.zeros((1,1))
Mean = np.concatenate((Mean,z),axis=0)*12
Mean[5,0] = 0.05 
print(Mean)

[[0.1135403 ]
 [0.10281814]
 [0.09179408]
 [0.08532523]
 [0.07845393]
 [0.05      ]]


In [15]:
## Value of Lambda
## We multiply the value of lambda by 2 

lambdada = 2 * 100

In [16]:
P_portfolio = lambdada * matrix(Cov) #Quadratic term in the objective function
print(P_portfolio)

[ 3.11e-01  3.73e-02  4.08e-02 -5.91e-02  7.00e-02  0.00e+00]
[ 3.73e-02  1.75e-01  5.49e-03  4.64e-02 -2.54e-02  0.00e+00]
[ 4.08e-02  5.49e-03  2.01e-01  3.18e-02  1.74e-02  0.00e+00]
[-5.91e-02  4.64e-02  3.18e-02  1.67e-01  1.60e-02  0.00e+00]
[ 7.00e-02 -2.54e-02  1.74e-02  1.60e-02  9.89e-02  0.00e+00]
[ 0.00e+00  0.00e+00  0.00e+00  0.00e+00  0.00e+00  0.00e+00]



In [17]:
#Linear term in the objective function
q_portfolio = matrix(Mean)

In [18]:
##Inequality constraints
G_portfolio = matrix([[-1,0,0,0,0,0],[0,-1,0,0,0,0],[0,0,-1,0,0,0],[0,0,0,-1,0,0],[0,0,0,0,-1,0],[0,0,0,0,0,-1]],tc='d')
h_portfolio = matrix([0.0,0.0,0.0,0.0,0.0,0.0])

#Equality constraints
A_portfolio = matrix([1.0,1.0,1.0,1.0,1.0,1.0], (1,6))
b_portfolio = matrix(1.0)

print(q_portfolio)
print(G_portfolio)
print(h_portfolio)
print(A_portfolio)
print(b_portfolio)

[ 1.14e-01]
[ 1.03e-01]
[ 9.18e-02]
[ 8.53e-02]
[ 7.85e-02]
[ 5.00e-02]

[-1.00e+00  0.00e+00  0.00e+00  0.00e+00  0.00e+00  0.00e+00]
[ 0.00e+00 -1.00e+00  0.00e+00  0.00e+00  0.00e+00  0.00e+00]
[ 0.00e+00  0.00e+00 -1.00e+00  0.00e+00  0.00e+00  0.00e+00]
[ 0.00e+00  0.00e+00  0.00e+00 -1.00e+00  0.00e+00  0.00e+00]
[ 0.00e+00  0.00e+00  0.00e+00  0.00e+00 -1.00e+00  0.00e+00]
[ 0.00e+00  0.00e+00  0.00e+00  0.00e+00  0.00e+00 -1.00e+00]

[ 0.00e+00]
[ 0.00e+00]
[ 0.00e+00]
[ 0.00e+00]
[ 0.00e+00]
[ 0.00e+00]

[ 1.00e+00  1.00e+00  1.00e+00  1.00e+00  1.00e+00  1.00e+00]

[ 1.00e+00]



In [19]:
MV = solvers.qp(P_portfolio, -q_portfolio, G_portfolio, h_portfolio, A_portfolio, b_portfolio) # Minimize the variance-mean of return
print(MV['x']) ##The optimal solution of the quadratic programming

     pcost       dcost       gap    pres   dres
 0: -6.8920e-02 -1.0794e+00  1e+00  1e-16  3e+00
 1: -6.8973e-02 -8.8831e-02  2e-02  8e-17  6e-02
 2: -6.9443e-02 -7.0858e-02  1e-03  8e-17  3e-03
 3: -6.9500e-02 -6.9545e-02  4e-05  2e-16  3e-05
 4: -6.9500e-02 -6.9500e-02  5e-07  8e-17  3e-07
 5: -6.9500e-02 -6.9500e-02  5e-09  7e-17  3e-09
Optimal solution found.
[ 1.34e-01]
[ 2.63e-01]
[ 1.33e-01]
[ 1.40e-01]
[ 2.15e-01]
[ 1.16e-01]



In [20]:
## Calculate the mean return and the standard deviation of return for the resulting portfolio

x_star = np.array(MV['x'])
MeanReturn = np.matmul(Mean.transpose(),x_star)

SDReturn = np.sqrt(np.matmul(np.matmul(x_star.transpose(),Cov),x_star))

print(MeanReturn)
print(SDReturn)


[[0.08899996]]
[[0.01396423]]


In [21]:
##Load the wine data
wine = pd.read_csv("wine.csv")
nrow = wine.shape[0]##Number of rows

In [22]:
wine.head()

,Year,Price,WinterRain,AGST,HarvestRain,Age,FrancePop
0,1952,7.4950,600,17.1167,160,31,43183.569
1,1953,8.0393,690,16.7333,80,30,43495.030
2,1955,7.6858,502,17.1500,130,28,44217.857
3,1957,6.9845,420,16.1333,110,26,45152.252
4,1958,6.7772,582,16.4167,187,25,45653.805


In [23]:
## Construct the data set 
## We do not use Year, since it is highly correlated with FrancePop

X = np.ones((nrow,1))#A colume of 1s

X0 = np.array(wine)
Y = X0[:,1].reshape((nrow,1))
X1 = X0[:,2:7]##Covariate matrix

#Attaching everything together
X = np.concatenate((X, X1), axis=1)
XX = np.concatenate((Y,X),axis=1)

print(XX)

[[7.4950000e+00 1.0000000e+00 6.0000000e+02 1.7116700e+01 1.6000000e+02
  3.1000000e+01 4.3183569e+04]
 [8.0393000e+00 1.0000000e+00 6.9000000e+02 1.6733300e+01 8.0000000e+01
  3.0000000e+01 4.3495030e+04]
 [7.6858000e+00 1.0000000e+00 5.0200000e+02 1.7150000e+01 1.3000000e+02
  2.8000000e+01 4.4217857e+04]
 [6.9845000e+00 1.0000000e+00 4.2000000e+02 1.6133300e+01 1.1000000e+02
  2.6000000e+01 4.5152252e+04]
 [6.7772000e+00 1.0000000e+00 5.8200000e+02 1.6416700e+01 1.8700000e+02
  2.5000000e+01 4.5653805e+04]
 [8.0757000e+00 1.0000000e+00 4.8500000e+02 1.7483300e+01 1.8700000e+02
  2.4000000e+01 4.6128638e+04]
 [6.5188000e+00 1.0000000e+00 7.6300000e+02 1.6416700e+01 2.9000000e+02
  2.3000000e+01 4.6583995e+04]
 [8.4937000e+00 1.0000000e+00 8.3000000e+02 1.7333300e+01 3.8000000e+01
  2.2000000e+01 4.7128005e+04]
 [7.3880000e+00 1.0000000e+00 6.9700000e+02 1.6300000e+01 5.2000000e+01
  2.1000000e+01 4.8088673e+04]
 [6.7127000e+00 1.0000000e+00 6.0800000e+02 1.5716700e+01 1.5500000e+02
 

In [24]:
## Calculate the mean squared error loss function, given beta and the training data

def Loss(beta,XX):
    Y = XX[:,0]
    X = XX[:,1:7]
    return 1/nrow*np.sum(np.square(Y-np.matmul(X,beta)))


In [25]:
## Calculate the Jacobian of the loss function

def Loss_jac(beta,XX):
    Y = XX[:,0]
    X = XX[:,1:7]
    return np.reshape(1/nrow*2*np.matmul(np.transpose(X),np.matmul(X,beta)-Y),(6,))

In [26]:
## Specify the initial search point

beta0 = np.zeros(6)
print(beta0)

[0. 0. 0. 0. 0. 0.]


In [27]:
## Specify the coefficient of the objective function (i.e., the loss function)

coeffs = XX
print(coeffs)

[[7.4950000e+00 1.0000000e+00 6.0000000e+02 1.7116700e+01 1.6000000e+02
  3.1000000e+01 4.3183569e+04]
 [8.0393000e+00 1.0000000e+00 6.9000000e+02 1.6733300e+01 8.0000000e+01
  3.0000000e+01 4.3495030e+04]
 [7.6858000e+00 1.0000000e+00 5.0200000e+02 1.7150000e+01 1.3000000e+02
  2.8000000e+01 4.4217857e+04]
 [6.9845000e+00 1.0000000e+00 4.2000000e+02 1.6133300e+01 1.1000000e+02
  2.6000000e+01 4.5152252e+04]
 [6.7772000e+00 1.0000000e+00 5.8200000e+02 1.6416700e+01 1.8700000e+02
  2.5000000e+01 4.5653805e+04]
 [8.0757000e+00 1.0000000e+00 4.8500000e+02 1.7483300e+01 1.8700000e+02
  2.4000000e+01 4.6128638e+04]
 [6.5188000e+00 1.0000000e+00 7.6300000e+02 1.6416700e+01 2.9000000e+02
  2.3000000e+01 4.6583995e+04]
 [8.4937000e+00 1.0000000e+00 8.3000000e+02 1.7333300e+01 3.8000000e+01
  2.2000000e+01 4.7128005e+04]
 [7.3880000e+00 1.0000000e+00 6.9700000e+02 1.6300000e+01 5.2000000e+01
  2.1000000e+01 4.8088673e+04]
 [6.7127000e+00 1.0000000e+00 6.0800000e+02 1.5716700e+01 1.5500000e+02
 

In [28]:
## Minimize the square loss to fit a linear regression model

OLS = opt.minimize(Loss,beta0,jac=Loss_jac,args=coeffs,method='SLSQP',tol=1e-10)

In [29]:
## Solution to the optimization model

OLS.x

array([-4.50398864e-01,  1.04250681e-03,  6.01223884e-01, -3.95812450e-03,
        5.84748489e-04, -4.95273038e-05])

In [30]:
# Compare the optimization results with the fitted linear regression model

from sklearn import datasets, linear_model

OLS_model = linear_model.LinearRegression(fit_intercept=False)
OLS_model.fit(X,Y)
OLS_model.coef_

array([[-4.50398864e-01,  1.04250681e-03,  6.01223884e-01,
        -3.95812450e-03,  5.84748489e-04, -4.95273038e-05]])

In [31]:
## Variance of return for a given portfolio x

def Var(x):
    return np.matmul(np.matmul(np.transpose(x),Cov),x)

In [32]:
## Mean return of for a given portfolio x

def MR(x):
    return np.matmul(np.transpose(Mean),x)

In [33]:
## Negative mean-variance utility given a portfolio x
def MV(x,lam):
    return lam*Var(x)-MR(x)

In [34]:
# Jacobian of negative mean-variance utility given a portfolio x

def MV_jac(x,lam):
    return np.reshape(2*lam*np.matmul(Cov,x)-np.reshape(Mean,(6,)),(6,))

In [35]:
# Budget constraint

def Budget(x):
    return np.sum(x)-1

In [36]:
# Jacobian of budget constraint

def Budget_jac(x):
    return np.ones(6)

In [37]:
# Bounds for the portfolios: 0<=xi<=1
bnds = ((0,1),(0,1),(0,1),(0,1),(0,1),(0,1))

In [38]:
# Budget constraint 
cns = ({'type': 'eq', 'fun': Budget, 'jac': Budget_jac})

In [39]:
# Initial value xi=0 for i=1,2,3,4,5,6
x0 = np.zeros(6)

In [40]:
# specify the value of lambda and obtain the optimal solution
lam = 500
MV_utility = opt.minimize(fun=MV,x0=x0,jac=MV_jac,args=lam,bounds=bnds,constraints=cns,method='SLSQP',tol=1e-20)

In [41]:
# Optimal solution of the Mean-variance problem
MV_utility.x

array([0.02671433, 0.05269813, 0.02652276, 0.02800183, 0.04292823,
       0.82313472])

In [42]:
#Mean return

MR(MV_utility.x)

array([0.0578])

In [43]:
#Square root of return

np.sqrt(Var(MV_utility.x))

0.0027928472793703044

In [44]:
##Using for loop to calculate the objective/constraint functions and the Jacobians
def Var1(x):
    y = 0
    for i in range(6):
        for j in range(6):
            y = y + x[i]*x[j]*Cov[i,j]
    return y

In [45]:
##Using for loop to re-establish objective and constraint functions, and their Jacobians

def MR1(x):
    y = 0
    for i in range(6):
        y = y + x[i]*Mean[i]    
    return y

In [46]:
def MV1(x,lam):
    return lam*Var1(x)-MR1(x)

In [47]:
def MV1_jac(x,lam):
    y = np.zeros(6)
    for i in range(6):
        for j in range(6):
            y[i] = y[i] + lam * x[j] * 2*Cov[i,j] 
        y[i] = y[i] - Mean[i]     
    return y

In [48]:
def Budget1(x):
    y = 0
    for i in range(6):
        y = y + x[i]
    return y - 1

In [49]:
def Budget1_jac(x):
    return np.ones(6)

In [50]:
cns1 = ({'type': 'eq', 'fun': Budget1, 'jac': Budget1_jac})

In [51]:
## Re-run the optimization

lam = 500
MV_utility1 = opt.minimize(fun=MV1,x0=x0,jac=MV1_jac,args=lam,bounds=bnds,constraints=cns1,method='SLSQP',tol=1e-20)

In [52]:
MV_utility1.x

array([0.02671433, 0.05269813, 0.02652276, 0.02800183, 0.04292823,
       0.82313472])